# WINDNODE ABW - Scenario Analysis

<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" width="100" style="float: right">

__copyright__ 	= "© Reiner Lemoine Institut" <br>
__license__ 	= "GNU Affero General Public License Version 3 (AGPL-3.0)" <br>
__url__ 		= "https://www.gnu.org/licenses/agpl-3.0.en.html" <br>
__author__ 		= "Julian Endres" <br>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
######## WINDNODE ###########
# define and setup logger
from windnode_abw.tools.logger import setup_logger
logger = setup_logger()

# load configs
from windnode_abw.tools import config
config.load_config('config_data.cfg')
config.load_config('config_misc.cfg')

from windnode_abw.analysis import analysis
from windnode_abw.tools.draw import *

######## DATA ###########

import re
import pandas as pd

######## PostgreSQL ###########
import sys
import os
import getpass
from sqlalchemy import *

######## Plotting ###########

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

# set seaborn style
sns.set()

import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go
import plotly.offline as pltly
from plotly.subplots import make_subplots


In [ ]:
# specify what to import (in path ~/.windnode_abw/)
run_timestamp = '2020-07-17_224437_1month'

# select multiple scenarios manually or use 'ALL' to analyze all
#scenarios = ['StatusQuo_DSMno','NEP2035']#,'ISE2050']
scenarios = ['StatusQuo_DSMno','NEP2035']
# run analysis
regions_scns, results_scns = analysis(run_timestamp=run_timestamp,
                                      scenarios=scenarios)


# TODO
- [ ] parallelize analysis
- [ ] save / pickle results

### useful params

In [ ]:
MUN_NAMES = regions_scns['NEP2035'].muns.gen.to_dict()

# General Information

## Installed Capacities

In [ ]:
df_data = results_scns['NEP2035']['parameters']['Installed capacity electricity supply']
df_data = df_data.rename(columns=PRINT_NAMES)

fig, axes = plt.subplots(3,3, figsize=(12,10))

for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    
    plot_geoplot(key, data, regions_scns['NEP2035'], ax=ax, unit='MW')
    
    
fig.suptitle('Installed el. Generation Capacity',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## Electrical Demand

In [ ]:
df_data = results_scns['NEP2035']['flows_txaxt']['Stromnachfrage'].sum(level=1)
df_data = df_data.drop(columns='export')
df_data.index = df_data.index.astype(int)
df_data = df_data.rename(columns=PRINT_NAMES)
df_data = df_data / 1e3

fig, axes = plt.subplots(1,3, figsize=(14,4))

for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    
    plot_geoplot(key, data, regions_scns['NEP2035'], ax=ax, unit='GWh')
    
    
fig.suptitle('Electrical Demand per Sector',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## Thermal Demand

In [ ]:
df_data = results_scns['NEP2035']['flows_txaxt']['Wärmenachfrage'].sum(level=2)
df_data.index = df_data.index.astype(int)
df_data = df_data.rename(columns=PRINT_NAMES)
df_data = df_data / 1e3

fig, axes = plt.subplots(1,3, figsize=(14,4))

for ax, (key, data) in  zip(axes.flat, df_data.items()):
    
    plot_geoplot(key, data, regions_scns['NEP2035'], ax=ax, unit='GWh')
    
    
fig.suptitle('Thermal Demand per Sector',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

# 3 Area required
- [x] absolute chloropleth
- [x] relative barplots

## absolute chloropleth

In [ ]:
df_data = results_scns['NEP2035']['results_axlxt']['Area required']
df_data = df_data.rename(columns=PRINT_NAMES)

fig, axes = plt.subplots(2,2, figsize=(12,6))

for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    
    plot_geoplot(key, data, regions_scns['NEP2035'], ax=ax, unit='ha')
    
    
fig.suptitle('Required Area',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

## relative barplots

In [ ]:
df_data = results_scns['NEP2035']['highlevel_results']

fig = go.Figure()

# PV rooftop
mask = [i for i in df_data.index if 'rooftop' in  i[0]]
data = df_data.loc[mask]
index = data.index.get_level_values(level=0)

fig.add_trace(
    go.Bar(y=index, x=data.values,  orientation='h', name='PV rooftop'))

# PV Ground
mask = [i for i in df_data.index if 'PV ground' in  i[0]]
data = df_data.loc[mask]
index = data.index.get_level_values(level=0)

fig.add_trace(
    go.Bar(y=index, x=data.values,  orientation='h', name='PV Ground'))#, visible='legendonly'))


# Wind
mask = [i for i in df_data.index if 'rel. wind' in  i[0]]
data = df_data.loc[mask]
index=  data.index.get_level_values(level=0)

fig.add_trace(
    go.Bar(y=index, x=data.values,  orientation='h', name='Wind'))#, visible='legendonly'))


fig.update_layout(title_text = 'Relative Required Area',
                  xaxis=dict(title=' %',
                    titlefont_size=12),
                    autosize=True)

fig.show()

## 4,5 Eigenversorgungsanteil

- [x] Absolut (Bilanz Jahresmenge) 2bars supply/demand
- [x] relativ in Prozent
- [ ] histogramm timeseries
- [ ] boxplots timeseries (prozentual Energiebilanz)

## one scenario

In [ ]:
df_data = results_scns['NEP2035']['results_axlxt']['Autarky']
df_data = df_data.rename(index=MUN_NAMES)
df_data = df_data.sort_values(axis=0, by='relative')
fig = go.Figure()



data = df_data['supply'] / 1e3

fig.add_trace(
    go.Bar(x=data.index, y=data.values,  orientation='v', name='Supply',yaxis="y1"))


data = df_data['demand'] / 1e3

fig.add_trace(
    go.Bar(x=data.index, y=data.values,  orientation='v', name='Demand',yaxis="y1"))


data = df_data['relative'] * 1e2

fig.add_trace(
    go.Bar(x=data.index, y=data.values,  orientation='v', name='Relative', visible='legendonly', yaxis="y2"))


fig.update_layout(title_text = 'Autarky per AGS',
                    autosize=True,
                  yaxis=dict(title="absolute in GWh"),
                  yaxis2=dict(title="relative in %",
                              #type="log",
                              anchor="x",
                              overlaying="y",
                              side="right",
                             )
                 )

fig.show()

## multiple scenarios

In [ ]:
df_data = pd.DataFrame({key: results_scns[key]['results_axlxt']['Autarky']['relative']
                        for key in results_scns.keys()})
df_data = df_data.rename(index=MUN_NAMES)
df_data = df_data.sort_values(axis=0, by=df_data.columns[0], ascending=False)
df_data = df_data * 1e2

fig = go.Figure()

for key, data in df_data.items():

    fig.add_trace(
        go.Bar(x=data.index, y=data.values,  orientation='v', name=key))#, visible='legendonly'))


fig.update_layout(title_text = 'Relative Autarky per AGS',
                    autosize=True,
                  yaxis=dict(title="relative in %"),
                 )

fig.show()

## split hbar

In [ ]:

df_data = results_scns['NEP2035']['results_axlxt']['Autarky']['relative']
df_data = df_data.rename(index=MUN_NAMES)
df_data = df_data.mul(100)
df_data = df_data.sort_values()

data=df_data.round()

limit=120

# split data
data_left = data[data < limit]
data_right = data[data >= limit]

fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.2)

fig.add_trace(
    go.Bar(y=data.index, x=data_left.values,  orientation='h', name=f'< {limit}'),
    row=1, col=1)

fig.add_trace(
    go.Bar(y=data.index, x=data_right.values,  orientation='h', name=f'> {limit}'),
    row=1, col=2)

fig.update_yaxes(type='category', row=1, col=1)
fig.update_yaxes(type='category', row=1, col=2)

fig.update_layout(title_text = 'relative Autarky',
                  xaxis=dict(title=' %',
                    titlefont_size=12),
                    autosize=True)

fig.show()

## pandas for multiple scenarios

In [ ]:
fig, axes  = plt.subplots(2, figsize=(15,10))

for scn, ax in zip(results_scns.keys(), axes):

    df_data = results_scns[scn]['results_axlxt']['Autarky']['relative']
    df_data = df_data.rename(index=MUN_NAMES)
    df_data = df_data.mul(100)
    df_data = data.sort_values()
    
    plot_split_hbar(df_data, 120, ax, unit='%', title=scn)

fig.suptitle('relative Autarky',
     fontsize=16,
     fontweight='normal')

plt.show()

## relative time

TODO
- [x] prozentual
- [ ] Summe über alle Gemeinde als TS zusätzlich?
- [ ] relativer Zeitliche Anteil Autarky 10% aller stunden?? was habe ich hiermit gemeint?
- [x] Violinplot über Timeseries relativer Anteil
- [x] geoplot

In [ ]:
df_data = results_scns['NEP2035']['results_axlxt']['Autarky']['hours']
max_hours  = len(results_scns['NEP2035']['flows_txaxt']['Autarky'].index.get_level_values(level=0).unique())

df_data = df_data / max_hours * 100

df_data = df_data.sort_values(ascending=False)
df_data = df_data.rename(index=MUN_NAMES)

fig, ax = plt.subplots(1, figsize=(15,5))
df_data.plot(kind='bar', ax=ax)
ax.set_title('Percentage of autark hours',
            fontsize=16)
ax.set_ylabel('%')
ax.tick_params(axis='x', rotation=45)
plt.show()

- rechter Plot müsste eigentlich eine Prozentscala haben

### geoplot

In [ ]:
df_data = results_scns['NEP2035']['results_axlxt']['Autarky'].loc[:,['hours', 'relative']]
df_data = df_data.rename(columns=PRINT_NAMES)

fig, axes = plt.subplots(1,2, figsize=(12,5))

for ax, (key, data) in  zip(axes.flat, df_data.iteritems()):
    
    plot_geoplot(key, data, regions_scns['NEP2035'], ax=ax)
    
    
fig.suptitle('Autark hours and relative Autarky',
     fontsize=16,
     fontweight='normal')
plt.tight_layout()
plt.show()

### violinplot

In [ ]:
# violinplots
df_data = results_scns['NEP2035']['flows_txaxt']['Autarky']['relative'].unstack()

df_data.columns = [int(i) for i in df_data.columns]
df_data = df_data.rename(columns=MUN_NAMES)

limit = 2

fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.25)

for ags, data in df_data.iteritems():
    
    if data.describe().loc['mean'] > limit:

        fig.add_trace(go.Violin(x=data.values, name=ags, orientation='h'), row=1, col=1)
    else:
        fig.add_trace(go.Violin(x=data.values, name=ags,  orientation='h'), row=1, col=2)


fig.update_xaxes(title_text="% autarky", row=1, col=1)
fig.update_yaxes(type='category', row=1, col=1)
fig.update_yaxes(type='category', row=1, col=2)


fig.update_layout(
    title='Relative Autarky')
fig.show()

# 6 Energymix

INFO:
- export == ins nationale Netz
- ABW_... == intra_regional

TODO:

BUGS:
- [ ] supply und demand sind unausgeglichen!

In [ ]:
supply = results_scns['NEP2035']['flows_txaxt']['Stromerzeugung'].sum(level=1)
abw_import = results_scns['NEP2035']['flows_txaxt']['Intra-regional exchange']['import'].sum(level=1)
abw_import = abw_import.rename('ABW-import')
supply = supply.join(abw_import)


demand = results_scns['NEP2035']['flows_txaxt']['Stromnachfrage'].sum(level=1)
abw_export = results_scns['NEP2035']['flows_txaxt']['Intra-regional exchange']['export'].sum(level=1)
abw_export = abw_export.rename('ABW-export')
demand = demand.join(abw_export)
    
plot_snd_total(regions_scns['NEP2035'], supply , demand)

## Energieversorgungsmix - Verlauf

TODO:

- [ ] Stromspeicher?
- [ ] Flexibilitäten?

BUGS:

- [ ] intra regional exchanges always zero

In [ ]:
plot_timeseries(results_scns['NEP2035'], kind='el')#ags='15091160')

# 7 Emissions

TODO
- [ ] units? t CO2?
- [ ] emissions compared to SQ
- [ ] <s> emissions compared to 1990 (research TODO) </s>
- [ ] große Werte besonders schöne Farben
- [ ] Notiz zur Zuordnung von KWK zu el etc
- [ ] Legend sortieren

In [ ]:
df_data_left = results_scns['NEP2035']['results_t']['CO2 emissions th. total'].rename('th').to_frame()
df_data_right = results_scns['NEP2035']['results_t']['CO2 emissions el. total'].rename('el').to_frame()

# drop nans & zeros
df_data_left = df_data_left[df_data_left!=0].dropna()
df_data_right = df_data_right[df_data_right!=0].dropna()

df = df_data_left.join(df_data_right, how='outer')

df = df.T
df = df.sort_values(by=list(df.index), axis=1, ascending=True)

fig = px.bar(df, orientation='h', title='CO2 Emissions')

fig.update_layout(barmode='stack', xaxis={'categoryorder':'total ascending'})
fig.update_xaxes(title_text='t CO2')
fig.update_yaxes(title_text='')
fig.show()

# 12, LCOE

<div class="alert alert-block alert-info">
<b>Notes on LCOE calculation</b>

- Total LCOE calculate as $LCOE=\frac{expenses_{el.total}}{demand_{el.,total}}$, likewise total LCOH calculate as $LCOH=\frac{expenses_{th.,total}}{demand_{th.,total}}$
- Total expenses $expenses_{el.total}$ are annual expenses. Investment costs are discounted to one year using equivalent periodic costs
- The plot below shows fractions of these LCOE that are calculated as $LCOE_{technology}=\frac{expenses_{el.,technology}}{demand_{el.,total}}$ representation the share of each technology at total cost of one MWh
</div>
    
## compare scenarios
- [ ] Anmerkung bzgl Interpretation:
    - teure Technologie vs viel genutzte Technologie (Kostenanteil)

In [ ]:
val = 'LCOE'
df = pd.DataFrame([results_scns[sci]['results_t'][val] for sci in scenarios], index=scenarios)

df = df.rename(columns=PRINT_NAMES)

df = df.sort_values(by=scenarios, axis=1, ascending=True)

fig = px.bar(df, orientation='h', title=f'{val}')

fig.update_layout(barmode='stack',
                  xaxis={'categoryorder':'total ascending'},
                  legend={'traceorder':'reversed'}
                 )
fig.update_xaxes(title_text='€/MWh')
fig.update_yaxes(title_text='')
fig.show()

In [ ]:
val = 'LCOE'
df = pd.DataFrame([results_scns[sci]['results_t'][val] for sci in scenarios], index=scenarios)

df = df.rename(columns=PRINT_NAMES)

df = df.sort_values(by=scenarios, axis=1, ascending=True)

fig = px.bar(df, orientation='h', title=f'{val}')

fig.update_layout(barmode='stack',
                  xaxis={'categoryorder':'total ascending'},
                  legend={'traceorder':'reversed'}
                 )
fig.update_xaxes(title_text='€/MWh')
fig.update_yaxes(title_text='')
fig.show()

## compare LCOE, LCOH
- [ ] Legende Sortieren?! 

In [ ]:
values = ['LCOE','LCOH']

df = pd.DataFrame([results_scns['NEP2035']['results_t'][i] for i in values], index=values)

df = df.rename(columns=PRINT_NAMES)

df = df.sort_values(by=values, axis=1, ascending=True)

fig = px.bar(df, orientation='h', title='LCOE and LCOH')

fig.update_layout(barmode='stack', xaxis={'categoryorder':'total ascending'})
fig.update_xaxes(title_text='€/MWh')
fig.update_yaxes(title_text='')
fig.show()

# 9,10 Line Loadings

- [ ] Leitungskapazität? TODO Guido
    - Um nach Auslastung zu schauen?

In [ ]:
df_data = results_scns['NEP2035']['flows_txaxt']['Line loading'].max(level=1) * 100
df_data = df_data.sort_index(ascending=False)

fig = go.Figure() 
fig.add_bar(y=df_data.index,
            x=df_data.values,
            orientation='h',
            name='line loading',
            marker_color='red')
fig.add_bar(y=df_data.index,
            x=100-df_data.values,
            orientation='h',
            name='free capacity',
            marker_color='green')

fig.update_layout(barmode="relative",
    title='Maximum Line Loading',
    yaxis_tickfont_size=12,
    xaxis=dict(
        title='Loading in %',
        titlefont_size=16,
        tickfont_size=12,
    ))

fig.update_yaxes(type='category')
fig.update_layout(width=800)
fig.show()


# ENDE

# advanced/split violin plot

# 8 Flexibility

## Heatstorage

Fragen:
- ist usage überhaupt der Nutzungsgrad? jain

- [ ] Kennzahlen erstellen
    - Charge / Ladeleistung (nominal value)
    - Discharge / Entladeleistung
    - Charge / Nennspeicherkapazität
    - Discharge / Nennspeicherkapazität

## Batterystorage

- Batteriespeicher leer?

- nur increase activation?

## DSM

- Bezug auf hh-demand-mit-dsm beziehen?
    - auf total
    - auf hh-demand
- absolute dsm activation
- pro AGS

- nur increase activation? JA!

# inter ags elec transport / chord diagramm

## chord diagram

- Plotly lässt sich nicht als bereits berechnetes HTML anzeigen / nur zur veranschaulichung
- -> Binder

#### Darstellung mit matplotlib

- Summe über Gemeinde als TS zusätzlich
- relativer Zeitliche Anteil Autarky
- mehrere Plots für ags?
- Summen über Tage?

- geoplot